In [3]:
import pandas as pd
import numpy as np
import glob
import time

from functools import reduce
from scipy import stats

In [4]:
path = r"C:\Users\Roy\Desktop\ForecastTemperature"

def read_ahu_one():

    ahu_one = "\AHU1\\"

    exh = pd.read_csv(path + ahu_one + "ahu1_evac.csv")
    ext = pd.read_csv(path + ahu_one + "ahu1_ext.csv")
    hum = pd.read_csv(path + ahu_one + "ahu1_hum.csv")
    sup = pd.read_csv(path + ahu_one + "ahu1_in.csv")
    rec = pd.read_csv(path + ahu_one + "ahu1_rec.csv")

    exh.columns = ['date', 'exh'] # Exhaust temperature
    ext.columns = ['date', 'ext'] # Outdoor temperature
    hum.columns = ['date', 'hum'] # Humidity
    sup.columns = ['date', 'sup'] # Supply temperature
    rec.columns = ['date', 'rec'] # Recirculation temperature

    return exh, ext, hum, sup, rec

evac, ext, hum, sup, rec = read_ahu_one()

def MergeData():

    list_of_data = [evac, ext, hum, sup, rec]

    data = reduce(lambda  left,right: pd.merge(left,right,on=['date'],
                                            how='outer'), list_of_data)

    # Reorder columns.
    data = data[['date', 'exh', 'ext', 'hum', 'rec', 'sup']]

    return data

data = MergeData()

def PrepareData():

    evac, ext, hum, sup, rec = read_ahu_one()

    data = MergeData()

    return data

data = PrepareData()

In [5]:
def CalculateEnergy(data):
    """Calculates the energy necessary to
    attain a certain temperature."""

    # Creates two new columns: energy and sup_diff.
    # Energy will be calculated using specific heat.
    # Sup_diff calculates the difference in supply air temp.
    data['energy'] = None
    data['energy'] = pd.to_numeric(data['energy'], downcast='integer')
    data['sup_diff'] = data['sup'].diff()
    data['sup_diff'][0] = "34.6"

    # The approximate density of air (room temperature) is 1.292 per cubic meter (m3).
    # The cubic meter per hour in Cabin J14 is 439 m3/h.
    mass = (1.292 * 439) # Kilograms.
    specific_heat = 1012 # Joule.

    start = time.time()

    for row in data.itertuples():

        # Calculates the energy required to heat up a room.
        data['energy'] = (mass * specific_heat * data['sup_diff']).round(decimals=2)
        data['kwh'] = (data['energy'] * 0.0000002778)
        
        # Fills up zero values with the average value.
        data['energy'] = data['energy'].replace(to_replace=0, value=data['energy'].mean())

    print(data.head(n=15))

    # Sums up J and converts it to kWh.
    JouleSum = data['energy'].sum()
    kWhSum = (JouleSum * 0.0000002778)
    
    print("\nTotal energy use is {0:,.2f} in Joules.".format(JouleSum))
    print("This is {0:,.2f} in kWh.".format(kWhSum))

    stop = time.time()
    c_time = stop - start

    print("This computation took " + str((c_time / 60)) + " minutes.")

    return data

data = CalculateEnergy(data)

                        date   exh  ext   hum   rec   sup        energy  \
0   06-Jan-17 5:15:00 PM EET  27.2 -2.5  50.0  10.6  34.6  1.986020e+07   
1   06-Jan-17 5:20:00 PM EET  27.2 -2.5  50.0  10.6  34.7  5.739943e+04   
2   06-Jan-17 5:25:00 PM EET  27.2 -2.5  50.0  10.6  34.2 -2.869971e+05   
3   06-Jan-17 5:30:00 PM EET  27.2 -3.0  50.0  10.6  34.7  2.869971e+05   
4   06-Jan-17 5:35:00 PM EET  27.2 -3.5  50.0  10.6  34.2 -2.869971e+05   
5   06-Jan-17 5:40:00 PM EET  27.2 -3.5  50.0  10.1  34.2  1.599319e+02   
6   06-Jan-17 5:45:00 PM EET  27.2 -3.5  50.0  10.1  34.2  1.599319e+02   
7   06-Jan-17 5:50:00 PM EET  27.2 -3.5  50.0  10.1  34.2  1.599319e+02   
8   06-Jan-17 5:55:00 PM EET  27.2 -3.5  50.0  10.1  34.7  2.869971e+05   
9   06-Jan-17 6:00:00 PM EET  27.2 -4.0  50.0  10.1  34.7  1.599319e+02   
10  06-Jan-17 6:05:00 PM EET  27.2 -4.0  50.0  10.1  34.7  1.599319e+02   
11  06-Jan-17 6:10:00 PM EET  27.2 -3.5  50.0  10.1  34.7  1.599319e+02   
12  06-Jan-17 6:15:00 PM 

In [6]:
def CalculateRMT(data):
    """This function calculates the
    hourly running mean outdoor and 
    indoor temperatures and humidity."""
    
    data = data.set_index('date')
    data.index = pd.to_datetime(data.index, utc=True)

    # Resamples each variable into an hourly mean.
    hourly_mean_outside = data['ext'].resample('H').mean()
    hourly_mean_supply = data['sup'].resample('H').mean()
    hourly_mean_humidity = data['hum'].resample('H').mean()

    # Appends variables as fresh columns.
    data['rmot'] = hourly_mean_outside
    data['rmit'] = hourly_mean_supply
    data['rmoh'] = hourly_mean_humidity

    columns = ['rmot', 'rmit', 'rmoh']

    data[columns] = data[columns].fillna(method='ffill')
    data[columns] = data[columns].fillna(method='bfill')
    data[columns] = data[columns].round(decimals=2)
    
    print(data.head(n=10))

    return data

data = CalculateRMT(data)

                            exh  ext   hum   rec   sup        energy  \
date                                                                   
2017-01-06 17:15:00+00:00  27.2 -2.5  50.0  10.6  34.6  1.986020e+07   
2017-01-06 17:20:00+00:00  27.2 -2.5  50.0  10.6  34.7  5.739943e+04   
2017-01-06 17:25:00+00:00  27.2 -2.5  50.0  10.6  34.2 -2.869971e+05   
2017-01-06 17:30:00+00:00  27.2 -3.0  50.0  10.6  34.7  2.869971e+05   
2017-01-06 17:35:00+00:00  27.2 -3.5  50.0  10.6  34.2 -2.869971e+05   
2017-01-06 17:40:00+00:00  27.2 -3.5  50.0  10.1  34.2  1.599319e+02   
2017-01-06 17:45:00+00:00  27.2 -3.5  50.0  10.1  34.2  1.599319e+02   
2017-01-06 17:50:00+00:00  27.2 -3.5  50.0  10.1  34.2  1.599319e+02   
2017-01-06 17:55:00+00:00  27.2 -3.5  50.0  10.1  34.7  2.869971e+05   
2017-01-06 18:00:00+00:00  27.2 -4.0  50.0  10.1  34.7  1.599319e+02   

                           sup_diff       kwh  rmot  rmit  rmoh  
date                                                             
201

In [7]:
def Interpolate(data):
    """Fills up more 0 values."""

    columns = ['exh', 'ext', 'hum', 'rec',
               'sup', 'rmot', 'rmit', 'rmoh']

    data[columns] = data[columns].replace(to_replace=0, value=data[columns].mean())
    data[columns] = data[columns].round(decimals=2)

    return data

datA = Interpolate(data)    

In [14]:
def RemoveOutlierZ(data):

    data['z'] = np.abs(stats.zscore(data['sup']))

    print("\nHighest z-score is " + str(data['z'].max()))

    print("\nData shape before outlier removal: " + str(data.shape))
    data = data[data.z < 2.5]
    print("Data shape after outlier removal: " + str(data.shape))
    z = np.abs(stats.zscore(data['sup']))
    print("\nHighest z-score is now " + str(data['z'].max()))

    return data

data = RemoveOutlierZ(datA)
data_v2 = pd.DataFrame.copy(data, deep=True)


Highest z-score is 4.595157665692344

Data shape before outlier removal: (103004, 12)
Data shape after outlier removal: (100278, 12)

Highest z-score is now 2.4949638670015575


In [15]:
data.to_csv('data.csv')